In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


In [4]:
import pandas as pd

df=pd.read_csv('data/Churn_Modelling.csv')

df.drop(columns=['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

x=df.drop('Exited',axis=1)
label_encoder=LabelEncoder()

x['Gender']=label_encoder.fit_transform(x['Gender'])

one_hot_encoder=OneHotEncoder()

geo_encoder=one_hot_encoder.fit_transform(x[['Geography']]).toarray()

geo_encoder=pd.DataFrame(geo_encoder,columns=one_hot_encoder.get_feature_names_out(['Geography']))

x=pd.concat((x.drop('Geography',axis=1),geo_encoder),axis=1)

y=df['Exited']

In [5]:
scaler=StandardScaler()

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [13]:
def create_model(neuron=32,layers=1):
    model=Sequential()
    model.add(Dense(neuron,activation='relu',input_shape=(x_train.shape[1],)))
    
    for _ in range(layers-1):
        model.add(Dense(neuron,activation='relu'))
        
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [14]:
model=KerasClassifier(layers=1,neuron=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [15]:
param_grid={
    'neuron':[16,32,64,128],
    'layers':[1,2,3],
    'epochs':[50,100]
}

In [16]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_Result=grid.fit(x_train,y_train)

print(f'best : {grid_Result.best_score_} using params {grid_Result.best_params_}')

e:\machine learning\project\ANN-classification\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
e:\machine learning\project\ANN-classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


best : 0.857124432964259 using params {'epochs': 50, 'layers': 2, 'neuron': 16}
